In [8]:
import pandas as pd
import openpyxl
from openpyxl.chart import BarChart, Reference, Series


# リストの乗算 [2023] * 11 を使い、効率的に列を定義
df_2023 = pd.DataFrame({
    '売上年': [2023] * 11,
    '商品': ['紅茶', 'ハチミツ', '水筒', 'コーヒー豆', 'セッケン', '紅茶', 'アロマキャンドル', 'ワイングラス', '包丁', 'フライパン', '水筒'],
    '金額（千円）': [1200, 1000, 250, 860, 1300, 670, 900, 850, 940, 670, 1150]
})
df_2023.to_excel('2023_年間売上表.xlsx', index=False)

# 2022年のデータ
try:
    df_2022 = pd.read_excel('2022_年間売上表.xlsx')
except FileNotFoundError:
    df_2022 = pd.DataFrame({
        '売上年': [2022] * 11,
        '商品': ['紅茶', 'ハチミツ', 'セッケン', '紅茶', 'アロマキャンドル', 'ハチミツ', 'ハチミツ', 'ワイングラス', 'アロマキャンドル', '包丁', '水筒'],
        '金額（千円）': [1200, 1000, 1300, 800, 900, 300, 560, 850, 810, 940, 1150]
    })
    df_2022.to_excel('2022_年間売上表.xlsx', index=False)


df_all = pd.concat([df_2022, df_2023], ignore_index=True)

grouped = df_all.groupby(['商品', '売上年']).agg({'金額（千円）': 'sum'}).reset_index()

comparison_table = grouped.pivot(index='商品', columns='売上年', values='金額（千円）').fillna(0)

writer = pd.ExcelWriter('売上集計表.xlsx')
comparison_table.to_excel(writer, sheet_name='集計結果')
writer.close()


book = openpyxl.load_workbook('売上集計表.xlsx')
sheet = book['集計結果']

chart_sheet = book.create_sheet('売上比較グラフ')

chart = BarChart()
chart.title = '年度別売上比較'
chart.y_axis.title = '金額（千円）'
chart.x_axis.title = '商品'

max_row = comparison_table.shape[0] + 1

data_2022 = Reference(sheet, min_col=2, min_row=1, max_row=max_row)
series_2022 = Series(data_2022, title='2022年')
chart.series.append(series_2022)

data_2023 = Reference(sheet, min_col=3, min_row=1, max_row=max_row)
series_2023 = Series(data_2023, title='2023年')
chart.series.append(series_2023)

category_ref = Reference(sheet, min_col=1, min_row=2, max_row=max_row)
chart.set_categories(category_ref)

chart_sheet.add_chart(chart, 'A1')

book.save('売上集計表.xlsx')

print("連結・集計・グラフ作成が完了しました。")

連結・集計・グラフ作成が完了しました。
